In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('Data/S1_freq.csv.gz', sep='\t', header=None, compression='gzip')

In [ ]:
df= data.iloc[:, 1:]
df.shape

In [80]:
# convert to numpy array
df2 = df.values

# convert to float
df2 = df.astype('float32')
 


df3 = df2.reshape(1, 17520251)

# remove 251 columns from the end
df3 = df3[:, :-251]

df3= df3.reshape(1000, 17520)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
dfA = pd.DataFrame(df3)
# get standard deviation of each column

# calculate the mean of each column
col_means = dfA.mean()

# subtract each column mean from its values
dfB = dfA.sub(col_means, axis=1)

# create a new DataFrame with the resulting values
dfB = pd.DataFrame(dfB)


In [ ]:
dfB

In [ ]:
import random
# create a dataframe
df4 = pd.DataFrame(df3)
countries = ['Hungary', 'Croatia', 'Jordan', 'France',
             'Kazakhstan', 'Iran', 'Denmark', 'Luxembourg',
             'Estonia', 'CzechRepublic', 'Italy', 'Sweden', 
             'United Kingdom', 'Armenia', 'Germany', 'Spain',
             'Switzerland', 'Latvia', 'Romania', 'The Netherlands', 
             'Turkey', 'Russia', 'Ukraine', 'Bulgaria', 'Poland', 
             'Austria', 'Georgia', 'Portugal', 'Israel',
             'Lithuania', 'Macedonia', 'Serbia', 'Greece', 'Norway']

age = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000]

# create 1000 random combinations of country_age
country= []
bp = []
for i in range(1000):
    country.append(random.choice(countries))
    bp.append(random.choice(age))
    
# add country and age to the beginning of the dataframe
df4.insert(0, 'country', country)
df4.insert(1, 'age', bp)



In [ ]:
df4

In [ ]:
# change column names to SNP 1-17520
df4.columns = ['country']+['age'] + ['SNP' + str(i) for i in range(1, 17521)]

In [ ]:
df4

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

def is_data_spherical(X):
    # Compute the centroid of the data
    centroid = np.mean(X, axis=0)

    # Calculate the Euclidean distances from each point to the centroid
    distances = euclidean_distances(X, [centroid]).ravel()

    # Compute the mean and standard deviation of the distances
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)

    # Determine if the data is spherical
    if std_distance / mean_distance < 0.5:
        return True
    else:
        return False

print(is_data_spherical(dfB))

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np

def plot_silhouette_score(X, k_min=1, k_max=33):
    # Create a range of K values
    k_range = range(k_min, k_max+1)

    # Create an empty list to store the silhouette scores
    silhouette_scores = []

    # Loop through the range of K values and calculate the silhouette score for each value
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=0)
        labels = kmeans.fit_predict(X)
        silhouette_avg = silhouette_score(X, labels)
        silhouette_scores.append(silhouette_avg)

    # Plot the silhouette scores for each K value
    plt.plot(k_range, silhouette_scores)
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette score')
    plt.title('Silhouette score for K-means clustering')
    plt.xticks(np.arange(k_min, k_max+1, step=2))
    plt.show()
    

plot_silhouette_score(dfB, k_min=2, k_max=33)



In [ ]:
from joblib import Parallel, delayed
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt


def calculate_inertia(X, k):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)
    return kmeans.inertia_


def plot_elbow(X, k_min=1, k_max=33, n_jobs=-1):
    # Create a range of K values
    k_range = range(k_min, k_max+1)

    # Calculate inertias for each value of K in parallel
    inertias = Parallel(n_jobs=n_jobs)(delayed(calculate_inertia)(X, k) for k in k_range)

    # Plot the inertias for each K value
    plt.plot(k_range, inertias)
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.title('Inertia for K-means clustering')
    plt.xticks(np.arange(k_min, k_max+1, step=2))
    plt.show()


plot_elbow(dfB, k_min=2, k_max=33, n_jobs=-1)


In [ ]:

# Scale the data
scaler = StandardScaler()

# from column 1 to the end
data2= scaler.fit_transform(dfB)

# Choose the number of clusters
k = 24
# Initialize K centroids randomly
kmeans = KMeans(n_clusters=k, init='random', n_init=10)

# Fit the K-means model to the data
kmeans.fit(dfB)

# Get the cluster assignments for each SNP
labels = kmeans.labels_

# Get the cluster centers
centroids = kmeans.cluster_centers_

# Get the sum of squared distances of samples to their closest cluster center
inertia = kmeans.inertia_

# from first column of data get the country names and store in a list
country = data.iloc[:, 0].to_list()

# create a dictionary with country as key and labels as value
country_labels = dict(zip(country, labels))


# Print the results
print('Cluster labels: %s' % country_labels )
print('Cluster centroids: %s' % centroids)
print('Sum of squared distances: %s' % inertia)

# Plot the results
import matplotlib.pyplot as plt
plt.scatter(data2[:, 0], data2[:, 1], c=labels, cmap='rainbow')
plt.show()


In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score


# Scale the data
scaler = StandardScaler()
X = scaler.fit_transform(dfA)

#

# Perform spectral clustering
n_clusters = 15 # Number of clusters
clustering = SpectralClustering(n_clusters=n_clusters, eigen_solver='arpack', affinity="nearest_neighbors").fit(X)

# evaluate the clustering result
print(silhouette_score(X, clustering.labels_))


# Visualize the clustering result
plt.scatter(X[:, 0], X[:, 1], c=clustering.labels_)
plt.title("Spectral Clustering ({} clusters)".format(n_clusters))
plt.show()


